In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import csv
import random
from math import sqrt
import sklearn.metrics as metrics

# INITIALIZATION
population_size = 1000
max_generation = 1000
_lambda = 1.5
dimension = 6
max_domain = 100
min_domain = 0
step_size_cons = 0.01
Pa = 0.3
x = []
y = []

# CALCULATING ACCURACY USING r2 SCORE (since this model is currently getting used as a regression model)
def accuracy(array, data, labels, data_size):
    y_predicted = []
    for i in range(data_size):
        y_predicted.append(np.dot(data[i], array))
    x_actual = labels
    acc = metrics.r2_score(x_actual, y_predicted)
    return acc

# OBJECTIVE FUNCTION
def rmse(array, data, labels, data_size):  # CONVERTING THIS FUNCTION INTO rmse FUNC OF ANN
    y_predicted = []
    for i in range(data_size):
        y_predicted.append(np.dot(data[i], array))
    x_actual = labels
    fitness = sqrt(metrics.mean_squared_error(x_actual, y_predicted))
    return fitness

# LEVY FLIGHT
def levy_flight(Lambda):
    sigma1 = np.power((math.gamma(1 + Lambda) * np.sin((np.pi * Lambda) / 2)) \
                      / math.gamma((1 + Lambda) / 2) * np.power(2, (Lambda - 1) / 2), 1 / Lambda)
    sigma2 = 1
    u = np.random.normal(0, sigma1, size=dimension)
    v = np.random.normal(0, sigma2, size=dimension)
    step = u / np.power(np.fabs(v), 1 / Lambda)

    return step

class Individual:
    def __init__(self, data, labels, data_size):
        self.__position = np.random.rand(dimension) * (max_domain - min_domain) + min_domain
        self.__fitness = rmse(self.__position, data, labels, data_size)

    def get_position(self):
        return self.__position

    def get_fitness(self):
        return self.__fitness

    def set_position(self, position):
        self.__position = position

    def set_fitness(self, fitness):
        self.__fitness = fitness

    def abandon(self, data, labels, data_size):
        # abandon some variables
        for i in range(dimension):
            p = np.random.rand()
            if p < Pa:
                self.__position[i] = np.random.rand() * (max_domain - min_domain) + min_domain
        self.__fitness = rmse(self.__position, data, labels, data_size)

def main():

    """
    #WRITING RANDOM TRAIN DATA
    train_data_size = 100
    row = [[random.randint(0,100), random.randint(0,100), random.randint(0, 100), 0] for i in range(train_data_size+1)]
    row[0] = ['INPUT1', 'INPUT2', 'INPUT3', 'OUTPUT']
    for i in range(1, train_data_size+1):
        row[i][3] = row[i][0]+row[i][1]+row[i][2]
    with open('train_data.csv', 'w') as cs0vFile:
        writer = csv.writer(csvFile)
        for i in range(train_data_size+1):
            writer.writerow(row[i])
    csvFile.close()
    
    
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)   """

    # READING TRAIN DATA
t_data = pd.read_csv("C:\\Users\\gautam vijh\\Desktop\\minor project\\dataset\\dwarka\\PollutionTrainA.csv").dropna()
phase1 = t_data.drop(['Ammonia'], axis=1) # DROPING OUTPUT COLUMN FROM DATA SET
phase2 = phase1.values                   # CONVERTING THE WHOLE DATA INTO NUMPY ARRAY IN ROW MAJOR
train_data = phase2.tolist()             # CONVERTING THE 2D NUMPY ARRAY INTO LIST
labels = t_data['Ammonia'].tolist()       # EXTRACTING LABELS FROM DATA SET
train_data_size = len(t_data)
    # print(train_data)

    # RANDOMLY CREATING HOSTS
cs_list = []
for i in range(population_size):
    cs_list.append(Individual(train_data, labels, train_data_size))


    # SORT TO GET THE BEST FITNESS
cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

best_fitness = cs_list[0].get_fitness()
best_position = cs_list[0].get_position()

fig = plt.figure()

    # INITIAL POPULATION DISTRIBUTION
ax1 = fig.add_subplot(131)
for i in range(population_size):
        ax1.scatter([cs_list[i].get_position()[0]], [cs_list[i].get_position()[1]])
ax1.set_title('Initial Population Distributtion')
ax1.set_xlabel('x-axis')
ax1.set_ylabel('y-axis')

ax3 = fig.add_subplot(133)

t = 1
while t < max_generation:

        # GENERATING NEW SOLUTIONS
        for i in range(population_size):

            # CHOOSING A RANDOM CUCKOO (say i)
            i = np.random.randint(low=0, high=population_size)

            # SETTING ITS POSITION USING LEVY FLIGHT
            position = (cs_list[i].get_position())+(step_size_cons*levy_flight(_lambda))

            # Simple Boundary Rule
            for i in range(dimension):
                if position[i] > max_domain:
                    position[i] = max_domain
                if position[i] < min_domain:
                    position[i] = min_domain

            cs_list[i].set_position(position)
            cs_list[i].set_fitness(rmse(cs_list[i].get_position(), train_data, labels, train_data_size))

            # CHOOSING A RANDOM HOST (say j)
            j = np.random.randint(0, population_size)
            while j == i:  # random id[say j] ≠ i
                j = np.random.randint(0, population_size)

            # RELAXATION
            if cs_list[j].get_fitness() > cs_list[i].get_fitness():
                cs_list[j].set_position(cs_list[i].get_position())
                cs_list[j].set_fitness(cs_list[i].get_fitness())

        # SORT (to Keep Best)
        cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

        # ABANDON SOLUTION (exclude the best)
        for a in range(1, population_size):
            r = np.random.rand()
            if (r < Pa):
                cs_list[a].abandon(train_data, labels, train_data_size)

        # RANKING THE CS LIST
        cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

        # FIND THE CURRENT BEST
        if cs_list[0].get_fitness() < best_fitness:
            best_fitness = cs_list[0].get_fitness()
            best_position = cs_list[0].get_position()

        # PRINTING SOLUTION IN EACH ITERATION
        print("iteration =", t, " best_fitness =", best_fitness)

        # FITNESS ARRAY
        x.append(t)
        y.append(best_fitness)

        t += 1

    # FITNESS PLOTTING
ax3.plot(x, y)

    # OPTIMIZED WEIGHTS
print("\nOptimized weights are ", *best_position)

   

    # READING TEST DATA
test_data = pd.read_csv("C:\\Users\\gautam vijh\\Desktop\\minor project\\dataset\\dwarka\\PollutionTestA.csv").dropna()
td = test_data.drop(['Ammonia'], axis=1).values.tolist()
test_labels = test_data['Ammonia'].tolist()
test_data_size = len(test_data)
    # print(test_data['INPUT1'][0])


print("RMSE of of final ANN is ", rmse(best_position, td, test_labels, test_data_size))
print("Accuracy of final ANN is ", accuracy(best_position, td, test_labels, test_data_size))

    
    # GRAPH FOR FITNESS
ax3.set_title('Fitness Curve')
ax3.set_xlabel('x-axis')
ax3.set_ylabel('y-axis')

    # FINAL POPULATION DISTRIBUTION
ax2 = fig.add_subplot(132)
for i in range(population_size):
        ax2.scatter([cs_list[i].get_position()[0]], [cs_list[i].get_position()[1]])
ax2.set_title('Final Population Distributtion after '+str(t)+' iterations')
ax2.set_xlabel('x-axis')
ax2.set_ylabel('y-axis')

    # SHOWING GRAPH
plt.show()


if __name__ == "__main__":
    main()

iteration = 1  best_fitness = 34.42420382424125
iteration = 2  best_fitness = 34.42420382424125
iteration = 3  best_fitness = 25.781681662611476
iteration = 4  best_fitness = 25.410921595890237
iteration = 5  best_fitness = 24.67243517100245
iteration = 6  best_fitness = 19.81416001507003
iteration = 7  best_fitness = 19.470702246245335
iteration = 8  best_fitness = 19.470702246245335
iteration = 9  best_fitness = 19.465774422245993
iteration = 10  best_fitness = 19.465774422245993
iteration = 11  best_fitness = 14.547762491271472
iteration = 12  best_fitness = 13.99934625996169
iteration = 13  best_fitness = 13.58183939297022
iteration = 14  best_fitness = 13.513879541762051
iteration = 15  best_fitness = 13.195397980650524
iteration = 16  best_fitness = 13.050100461940875
iteration = 17  best_fitness = 13.050100461940875
iteration = 18  best_fitness = 12.95465394907401
iteration = 19  best_fitness = 11.358196927419757
iteration = 20  best_fitness = 7.757884940359693
iteration = 21  b

iteration = 163  best_fitness = 0.16856986728533005
iteration = 164  best_fitness = 0.16856986728533005
iteration = 165  best_fitness = 0.16856986728533005
iteration = 166  best_fitness = 0.16856986728533005
iteration = 167  best_fitness = 0.16856986728533005
iteration = 168  best_fitness = 0.16856986728533005
iteration = 169  best_fitness = 0.16856986728533005
iteration = 170  best_fitness = 0.16856986728533005
iteration = 171  best_fitness = 0.16856986728533005
iteration = 172  best_fitness = 0.16856986728533005
iteration = 173  best_fitness = 0.16856986728533005
iteration = 174  best_fitness = 0.16856986728533005
iteration = 175  best_fitness = 0.16856986728533005
iteration = 176  best_fitness = 0.16856986728533005
iteration = 177  best_fitness = 0.16856986728533005
iteration = 178  best_fitness = 0.16856986728533005
iteration = 179  best_fitness = 0.16856986728533005
iteration = 180  best_fitness = 0.16856986728533005
iteration = 181  best_fitness = 0.16856986728533005
iteration = 

iteration = 321  best_fitness = 0.16856945351762423
iteration = 322  best_fitness = 0.16856945351762423
iteration = 323  best_fitness = 0.16856945351762423
iteration = 324  best_fitness = 0.16856945351762423
iteration = 325  best_fitness = 0.16856945351762423
iteration = 326  best_fitness = 0.16856945351762423
iteration = 327  best_fitness = 0.16856945351762423
iteration = 328  best_fitness = 0.16856945351762423
iteration = 329  best_fitness = 0.16856945351762423
iteration = 330  best_fitness = 0.16856945351762423
iteration = 331  best_fitness = 0.16856945351762423
iteration = 332  best_fitness = 0.16856945351762423
iteration = 333  best_fitness = 0.16856945351762423
iteration = 334  best_fitness = 0.16856945351762423
iteration = 335  best_fitness = 0.16856945351762423
iteration = 336  best_fitness = 0.16856945351762423
iteration = 337  best_fitness = 0.16856945351762423
iteration = 338  best_fitness = 0.16856945351762423
iteration = 339  best_fitness = 0.16856945351762423
iteration = 

iteration = 479  best_fitness = 0.16856945351762423
iteration = 480  best_fitness = 0.16856736446124704
iteration = 481  best_fitness = 0.16856736446124704
iteration = 482  best_fitness = 0.16856736446124704
iteration = 483  best_fitness = 0.16856736446124704
iteration = 484  best_fitness = 0.16856736446124704
iteration = 485  best_fitness = 0.16856736446124704
iteration = 486  best_fitness = 0.16856736446124704
iteration = 487  best_fitness = 0.16856736446124704
iteration = 488  best_fitness = 0.16856736446124704
iteration = 489  best_fitness = 0.1685670523812035
iteration = 490  best_fitness = 0.1685670523812035
iteration = 491  best_fitness = 0.1685670523812035
iteration = 492  best_fitness = 0.1685670523812035
iteration = 493  best_fitness = 0.1685670523812035
iteration = 494  best_fitness = 0.1685670523812035
iteration = 495  best_fitness = 0.1685670523812035
iteration = 496  best_fitness = 0.1685670523812035
iteration = 497  best_fitness = 0.1685670523812035
iteration = 498  best

iteration = 640  best_fitness = 0.1685670523812035
iteration = 641  best_fitness = 0.1685670523812035
iteration = 642  best_fitness = 0.1685670523812035
iteration = 643  best_fitness = 0.1685670523812035
iteration = 644  best_fitness = 0.1685670523812035
iteration = 645  best_fitness = 0.1685670523812035
iteration = 646  best_fitness = 0.1685670523812035
iteration = 647  best_fitness = 0.1685670523812035
iteration = 648  best_fitness = 0.1685670523812035
iteration = 649  best_fitness = 0.1685670523812035
iteration = 650  best_fitness = 0.1685670523812035
iteration = 651  best_fitness = 0.1685670523812035
iteration = 652  best_fitness = 0.1685670523812035
iteration = 653  best_fitness = 0.1685670523812035
iteration = 654  best_fitness = 0.1685670523812035
iteration = 655  best_fitness = 0.1685670523812035
iteration = 656  best_fitness = 0.1685670523812035
iteration = 657  best_fitness = 0.1685670523812035
iteration = 658  best_fitness = 0.1685670523812035
iteration = 659  best_fitness =

iteration = 801  best_fitness = 0.1685670523812035
iteration = 802  best_fitness = 0.1685670523812035
iteration = 803  best_fitness = 0.1685670523812035
iteration = 804  best_fitness = 0.1685670523812035
iteration = 805  best_fitness = 0.1685670523812035
iteration = 806  best_fitness = 0.1685670523812035
iteration = 807  best_fitness = 0.1685670523812035
iteration = 808  best_fitness = 0.1685670523812035
iteration = 809  best_fitness = 0.1685670523812035
iteration = 810  best_fitness = 0.1685670523812035
iteration = 811  best_fitness = 0.1685670523812035
iteration = 812  best_fitness = 0.1685670523812035
iteration = 813  best_fitness = 0.1685670523812035
iteration = 814  best_fitness = 0.1685670523812035
iteration = 815  best_fitness = 0.1685670523812035
iteration = 816  best_fitness = 0.1685670523812035
iteration = 817  best_fitness = 0.1685670523812035
iteration = 818  best_fitness = 0.1685670523812035
iteration = 819  best_fitness = 0.1685670523812035
iteration = 820  best_fitness =

iteration = 962  best_fitness = 0.1685670523812035
iteration = 963  best_fitness = 0.1685670523812035
iteration = 964  best_fitness = 0.1685670523812035
iteration = 965  best_fitness = 0.1685670523812035
iteration = 966  best_fitness = 0.1685670523812035
iteration = 967  best_fitness = 0.1685670523812035
iteration = 968  best_fitness = 0.1685670523812035
iteration = 969  best_fitness = 0.1685670523812035
iteration = 970  best_fitness = 0.1685670523812035
iteration = 971  best_fitness = 0.1685670523812035
iteration = 972  best_fitness = 0.1685670523812035
iteration = 973  best_fitness = 0.1685670523812035
iteration = 974  best_fitness = 0.1685670523812035
iteration = 975  best_fitness = 0.1685670523812035
iteration = 976  best_fitness = 0.1685670523812035
iteration = 977  best_fitness = 0.1685670523812035
iteration = 978  best_fitness = 0.1685670523812035
iteration = 979  best_fitness = 0.1685670523812035
iteration = 980  best_fitness = 0.1685670523812035
iteration = 981  best_fitness =

<Figure size 640x480 with 3 Axes>